<a id="top"></a>
# 03_TRAIN_CONTINUAL — Entrenamiento Continual con *presets*

**Qué hace este notebook**  
Entrena y evalúa modelos en **aprendizaje continual** (secuencia de tareas) usando una **configuración unificada** desde `configs/presets.yaml`. Permite:  
1) lanzar un *run* base con el método del preset,  
2) **comparar métodos** manteniendo fijos datos/modelo, y  
3) generar un **resumen agregado** de resultados en `outputs/summary/`.


---

## 🎯 Objetivos
- Centralizar la configuración de **modelo**, **datos/codificación temporal**, **optimizador** y **método continual** vía `presets.yaml`.
- Soportar **H5 offline** (si `use_offline_spikes: true`) o **CSV + codificación en runtime** (si `encode_runtime: true`), seleccionándolo de forma coherente con el preset.
- Comparar métodos (`naive`, `ewc`, `rehearsal`, `rehearsal+ewc`, y los bio-inspirados previstos: `as-snn`, `sa-snn`, `sca-snn`, `colanet`) con **idéntica preparación de datos**.
- Exportar un **CSV de agregados** con métricas clave (MAE/MSE por tarea, olvido absoluto/relativo, etc.).

## ✅ Prerrequisitos
- Haber generado `data/processed/tasks.json` (y opcionalmente `tasks_balanced.json`) con **01_DATA_QC_PREP** o **01A_PREP_BALANCED**.
- Si el preset usa **offline** (`use_offline_spikes: true`), haber creado los H5 compatibles con **02_ENCODE_OFFLINE** (mismo `encoder/T/gain/size/to_gray` que el preset).
- Revisar `configs/presets.yaml` (secciones `model`, `data`, `optim`, `continual`, `prep`).

## ⚠️ Notas importantes
- **No combines** `use_offline_spikes: true` y `encode_runtime: true`. El notebook lo detecta y lanza error.
- La **semilla** global se toma de `CFG["data"]["seed"]` para reproducibilidad.
- La carpeta de salida incluye en el nombre preset, método, *encoder*, modelo, *seed*, etc., para facilitar trazabilidad.

<a id="toc"></a>

## 🧭 Índice

- [1) Setup del entorno y paths](#sec-01)  
- [2) Carga del preset unificado (`configs/presets.yaml`)](#sec-02)  
- [3) Verificación de datos y selección de `tasks.json`](#sec-03)  
- [4) Factories DataLoaders + Modelo (+ tasks)](#sec-04)  
- [5) Ejecución base con el preset (eco de config + run)](#sec-06)  
- [6) Comparativa de métodos (mismo preset/semilla/datos)](#sec-07)  
- [7) Barrido de combinaciones (opcional)](#sec-08)  
- [8) Resumen completo: inventario → parseo → agregados → tabla](#sec-09)



<a id="sec-01"></a>
## 1) Setup del entorno y paths

**Objetivo**  
Preparar el entorno: limitar hilos BLAS (evitar *oversubscription*), detectar `ROOT` (raíz del repo) y añadirlo a `sys.path`, importar utilidades del proyecto y seleccionar dispositivo (`cuda` si está disponible). Se activan optimizaciones de PyTorch en GPU (TF32/cuDNN) para acelerar.

> Aquí **no** se leen aún los presets; solo se configura el runtime global. 

[↑ Volver al índice](#toc)

In [1]:
# =============================================================================
# Imports y setup de entorno (threads, paths, dispositivo)
# =============================================================================
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"

from pathlib import Path
import sys, json, torch

# Raíz del repo y sys.path
ROOT = Path.cwd().parents[0] if (Path.cwd().name == "notebooks") else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

# Librerías del proyecto
from src.datasets import ImageTransform, AugmentConfig
from src.models import build_model
from src.utils import load_preset

# Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Ajustes de rendimiento (opcional)
torch.set_num_threads(4)
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
torch.set_float32_matmul_precision("high")

print("Device:", device)

Device: cuda


<a id="sec-02"></a>

## 2) Carga del preset unificado (`configs/presets.yaml`)

**Objetivo**  
Cargar un **preset** (`fast` | `std` | `accurate`) y derivar toda la configuración operativa:

- **Modelo/transform**: tamaño de imagen, escala de grises, etc.
- **Datos/codificación**: `encoder` (`rate|latency|raw`), `T`, `gain`, `seed`.
- **DataLoader**: `num_workers`, `prefetch_factor`, `pin_memory`, `persistent_workers`.
- **Augment** opcional y **balanceo online** si procede.

Incluye un **guardarraíl**: si `use_offline_spikes: true` y `encode_runtime: true` están ambos activos, se aborta con un error claro (config inválida).  

[↑ Volver al índice](#toc)

In [2]:
# =============================================================================
# Config global: presets.yaml
# =============================================================================
from pathlib import Path
from src.datasets import ImageTransform, AugmentConfig
from src.utils import load_preset

PRESET = "fast"  # fast | std | accurate
CFG = load_preset(ROOT / "configs" / "presets.yaml", PRESET)

# ---- Modelo / Transform ------------------------------------------------------
MODEL_NAME = CFG["model"]["name"]
tfm = ImageTransform(
    CFG["model"]["img_w"],
    CFG["model"]["img_h"],
    to_gray=bool(CFG["model"]["to_gray"]),
    crop_top=None
)

# ---- Datos / codificación temporal ------------------------------------------
ENCODER = CFG["data"]["encoder"]
T       = int(CFG["data"]["T"])
GAIN    = float(CFG["data"]["gain"])
SEED    = int(CFG["data"]["seed"])

USE_OFFLINE_SPIKES = bool(CFG["data"].get("use_offline_spikes", False))
RUNTIME_ENCODE     = bool(CFG["data"].get("encode_runtime", False))

# ---- DataLoader / augment / balanceo ----------------------------------------
NUM_WORKERS = int(CFG["data"].get("num_workers") or 0)           # robusto ante None
PREFETCH    = int(CFG["data"].get("prefetch_factor") or 2)       # <- casteo robusto
PIN_MEMORY  = bool(CFG["data"].get("pin_memory", True))
PERSISTENT  = bool(CFG["data"].get("persistent_workers", True))

AUG_CFG = AugmentConfig(**(CFG["data"].get("aug_train") or {})) \
          if CFG["data"].get("aug_train") else None

USE_ONLINE_BALANCING = bool(CFG["data"].get("balance_online", False))
BAL_BINS = int(CFG["data"].get("balance_bins") or 50)
BAL_EPS  = float(CFG["data"].get("balance_smooth_eps") or 1e-3)

# Guardarraíles
if USE_OFFLINE_SPIKES and RUNTIME_ENCODE:
    raise RuntimeError("Config inválida: use_offline_spikes=True y encode_runtime=True a la vez.")

print(f"[PRESET={PRESET}] model={MODEL_NAME} {tfm.w}x{tfm.h} gray={tfm.to_gray}")
print(f"[DATA] encoder={ENCODER} T={T} gain={GAIN} seed={SEED}")
print(f"[LOADER] workers={NUM_WORKERS} prefetch={PREFETCH} pin={PIN_MEMORY} persistent={PERSISTENT}")
print(f"[BALANCE] online={USE_ONLINE_BALANCING} bins={BAL_BINS}")
print(f"[RUNTIME_ENCODE] {RUNTIME_ENCODE} | [OFFLINE_SPIKES] {USE_OFFLINE_SPIKES}")

[PRESET=fast] model=pilotnet_snn 200x66 gray=True
[DATA] encoder=rate T=10 gain=0.5 seed=42
[LOADER] workers=8 prefetch=2 pin=True persistent=True
[BALANCE] online=False bins=50
[RUNTIME_ENCODE] False | [OFFLINE_SPIKES] True


<a id="sec-03"></a>

## 3) Verificación de datos y selección de `tasks.json`

**Objetivo**  
Construir `task_list` y verificar que existen los *splits* por tarea:

- Si el preset pide **balanced** (`prep.use_balanced_tasks: true`) y existe `tasks_balanced.json`, se usa; en caso contrario, se cae a `tasks.json` (se informa).
- Se valida que `train/val/test.csv` existen para cada *run*.  
- Si entrenas con **H5 offline**, se comprueba que están presentes los H5 con **nomenclatura compatible** con el preset (`encoder/T/gain/size/to_gray`).

> Si falta algún H5 requerido, genera primero con **02_ENCODE_OFFLINE**.  

[↑ Volver al índice](#toc)

In [3]:
# =============================================================================
# Verificación de datos (splits y, si procede, H5)
# =============================================================================
from pathlib import Path as _P
import json

PROC = ROOT / "data" / "processed"

# --- Elegir tasks según el preset ---
USE_BALANCED = bool(CFG.get("prep", {}).get("use_balanced_tasks", False))
tb_name = (CFG.get("prep", {}).get("tasks_balanced_file_name") or "tasks_balanced.json")
t_name  = (CFG.get("prep", {}).get("tasks_file_name")           or "tasks.json")

cand_bal = PROC / tb_name
cand_std = PROC / t_name
TASKS_FILE = cand_bal if (USE_BALANCED and cand_bal.exists()) else cand_std

with open(TASKS_FILE, "r", encoding="utf-8") as f:
    tasks_json = json.load(f)

task_list = [{"name": n, "paths": tasks_json["splits"][n]} for n in tasks_json["tasks_order"]]
print("Usando:", TASKS_FILE.name)
for t in task_list:
    print(f" - {t['name']}: {_P(t['paths']['train']).name}")

# Guardarraíl: si se pidió balanced, exigir train_balanced.csv
if USE_BALANCED:
    for t in task_list:
        train_path = _P(tasks_json["splits"][t["name"]]["train"])
        if train_path.name != "train_balanced.csv":
            raise RuntimeError(
                f"[{t['name']}] Esperaba 'train_balanced.csv' en modo balanced, pero encontré '{train_path.name}'."
            )

# Si entrenas con H5 offline, comprueba que existan
if USE_OFFLINE_SPIKES:
    mw, mh = CFG["model"]["img_w"], CFG["model"]["img_h"]
    color = "gray" if CFG["model"]["to_gray"] else "rgb"
    gain_tag = (GAIN if ENCODER == "rate" else 0)
    missing = []
    for t in task_list:
        run = t["name"]
        base = PROC / run
        for split in ("train", "val", "test"):
            expected = base / f"{split}_{ENCODER}_T{T}_gain{gain_tag}_{color}_{mw}x{mh}.h5"
            if not expected.exists():
                missing.append(str(expected))
    if missing:
        print("[WARN] Faltan H5 compatibles con el preset.")
        print("       Genera primero con 02_ENCODE_OFFLINE.ipynb (o tools/encode_tasks.py).")

print("OK: verificación de splits.")
print(f"Preset en uso: {PRESET}")


Usando: tasks_balanced.json
 - circuito1: train_balanced.csv
 - circuito2: train_balanced.csv
OK: verificación de splits.
Preset en uso: fast


<a id="sec-04"></a>
## 4) Factories unificados: DataLoaders + Modelo (+ tasks)

**Objetivo**  
Crear, en una sola llamada, los **componentes coherentes con el preset**:

- `build_components_for(CFG, ROOT)` → devuelve `tfm`, `make_loader_fn`, `make_model_fn`.
  - El **loader** respeta automáticamente el modo datos (H5 offline vs. CSV+encode runtime), *workers/prefetch/pin/persistent*, *augment*, y **balanceo online** si está activo.
  - El **modelo** se instancia según `model.name` y parámetros asociados.
- `build_task_list_for(CFG, ROOT)` → devuelve `task_list` y el *tasks file* efectivamente usado.

> Con esto evitas duplicar lógica entre cuadernos y garantizas que **bench, entrenamiento y comparativa** usen la **misma** configuración.  

[↑ Volver al índice](#toc)


In [4]:
# === Factories y task list coherentes con el PRESET cargado ===
from src.utils import build_task_list_for, build_components_for

# Construye tfm, make_loader_fn y make_model_fn leyendo TODO de CFG (igual que hacías a mano):
tfm, make_loader_fn, make_model_fn = build_components_for(CFG, ROOT)

# Elige automáticamente tasks_balanced.json si el preset lo pide y existe; si no, tasks.json
task_list, tasks_file = build_task_list_for(CFG, ROOT)

print("Tasks file:", tasks_file.name)
print("make_loader_fn listo (H5 si use_offline_spikes=True; fallback CSV+runtime si no).")


Tasks file: tasks_balanced.json
make_loader_fn listo (H5 si use_offline_spikes=True; fallback CSV+runtime si no).


<a id="sec-05"></a>
## 5) Ejecución base con el preset (eco de config + run)

**Objetivo**  
Lanzar **un experimento** con el método y parámetros del preset (`CFG["continual"]`). La celda:

- Imprime un **resumen de configuración** (modelo, datos, loader, método).
- Ejecuta `run_continual(...)`.
- Guarda resultados en `outputs/continual_*` (incluye `continual_results.json` y `manifest.json` por tarea).

> Revisa la consola para confirmar dispositivo, *encoder/T/gain* y modo de datos (offline/ runtime).  

[↑ Volver al índice](#toc)

In [5]:
# =============================================================================
# Ejecución base con el preset (eco de config + run)
# =============================================================================
from src.runner import run_continual

# Echo de configuración “resumido” (lo esencial para el run)
print(f"[RUN] preset={PRESET} | method={CFG['continual']['method']} "
      f"| seed={CFG['data']['seed']} | enc={CFG['data']['encoder']} "
      f"| kwargs={CFG['continual'].get('params', {})}")
print(f"[MODEL] {MODEL_NAME} {tfm.w}x{tfm.h} gray={tfm.to_gray}")
print(f"[DATA] T={CFG['data']['T']} gain={CFG['data']['gain']} "
      f"| offline_spikes={CFG['data']['use_offline_spikes']} "
      f"| runtime_encode={CFG['data']['encode_runtime']}")
print(f"[LOADER] workers={CFG['data']['num_workers']} "
      f"prefetch={CFG['data']['prefetch_factor']} pin={CFG['data']['pin_memory']} "
      f"persistent={CFG['data']['persistent_workers']} "
      f"| aug={bool(CFG['data']['aug_train'])} "
      f"| balance_online={CFG['data']['balance_online']}")

out_path, _ = run_continual(
    task_list=task_list,
    make_loader_fn=make_loader_fn,   # wrapper (Celda 4)
    make_model_fn=make_model_fn,     # factory (Celda 5)
    tfm=tfm,
    cfg=CFG,                         # preset completo
    preset_name=PRESET,              # solo naming
    out_root=ROOT / "outputs",
    verbose=True,
)
print("OK:", out_path)

[RUN] preset=fast | method=ewc | seed=42 | enc=rate | kwargs={'lam': '3.0e8', 'fisher_batches': 800}
[MODEL] pilotnet_snn 200x66 gray=True
[DATA] T=10 gain=0.5 | offline_spikes=True | runtime_encode=False
[LOADER] workers=8 prefetch=2 pin=True persistent=True | aug=True | balance_online=False

--- Tarea 1/2: circuito1 | preset=fast | method=ewc_lam_3e+08 | B=128 T=10 AMP=True | enc=rate ---


Epoch 1/2:   0%|          | 0/196 [00:00<?, ?it/s]

[EWC] base=0.3973 | pen=0 | pen/base=0.000


Epoch 1/2:  52%|█████▏    | 101/196 [00:18<00:13,  6.80it/s]

[EWC] base=0.06017 | pen=0 | pen/base=0.000


...

Epoch 1/2:   1%|          | 1/85 [00:00<01:06,  1.27it/s]

[EWC] base=0.07175 | pen=0 | pen/base=0.000


Epoch 2/2:  20%|██        | 17/85 [00:02<00:07,  9.17it/s]

[EWC] base=0.09569 | pen=0.006409 | pen/base=0.067 | λ_actual=3.000e+08 → λ_sugerido≈4.479e+09 (target pen/base=1.0)


OK: /home/cesar/proyectos/TFM_SNN/outputs/continual_fast_ewc_lam_3e+08_lam_3e+08_rate_model-PilotNetSNN_66x200_gray_seed_42


<a id="sec-06"></a>
## 6) Comparativa de métodos (mismo preset / misma semilla / mismos datos)

**Objetivo**  
Ejecutar una **batería de métodos** cambiando **solo** `continual.method` y sus `params`, manteniendo fijos: preset, semilla, *encoder/T/gain*, tamaño de imagen, *augment*, etc.

- Se clona `CFG` por método y se invoca `run_continual(...)` con las **factories** del propio `cfg_i`.
- El diccionario `METHODS` puede ampliarse con nombres registrados en `src/methods/`:
  - `naive`, `ewc`, `rehearsal`, `rehearsal+ewc`
  - (bio-inspirados previstos) `as-snn`, `sa-snn`, `sca-snn`, `colanet`

**Recomendaciones**
- Si usas **offline H5**, asegúrate de que existen para el preset (`02_ENCODE_OFFLINE`).
- Si activas *replay* (rehearsal), puedes **reducir** `persistent_workers` para evitar atascos de DataLoader en algunos entornos; la celda ya lo ajusta como precaución.

[↑ Volver al índice](#toc)

In [6]:
# === COMPARATIVA DE MÉTODOS: mismo preset, misma semilla, mismos datos ===
from copy import deepcopy
from src.runner import run_continual
from src.utils import build_task_list_for, build_components_for

CFG_BASE = deepcopy(CFG)

METHODS = {
    #"naive": {},
    #"ewc": {"lam": 7e8, "fisher_batches": 500},
    #"ewc": {"lam": 1e9, "fisher_batches": 1000},
    "rehearsal": {"buffer_size": 3000, "replay_ratio": 0.1},
    "rehearsal+ewc": {"buffer_size": 3000, "replay_ratio": 0.1, "lam": 1e9, "fisher_batches": 1000},
    "as-snn": {"gamma_ratio": 0.3, "lambda_a": 1.59168, "ema": 0.824},
}

runs_out = []
for method_name, method_params in METHODS.items():
    cfg_i = deepcopy(CFG_BASE)
    cfg_i["continual"]["method"] = method_name
    cfg_i["continual"]["params"] = method_params
    if "rehearsal" in method_name:
        cfg_i["data"]["persistent_workers"] = False

    # (Re)construye factories por si el cfg cambia
    tfm_i, make_loader_fn_i, make_model_fn_i = build_components_for(cfg_i, ROOT)
    task_list_i, tasks_file_i = build_task_list_for(cfg_i, ROOT)

    print(f"\n=== RUN: preset={PRESET} | method={method_name} | seed={cfg_i['data']['seed']} "
          f"| enc={cfg_i['data']['encoder']} | kwargs={method_params} ===")
    out_dir, _ = run_continual(
        task_list=task_list_i,
        make_loader_fn=make_loader_fn_i,
        make_model_fn=make_model_fn_i,
        tfm=tfm_i,
        cfg=cfg_i,
        preset_name=PRESET,
        out_root=ROOT / "outputs",
        verbose=True,
    )
    runs_out.append(out_dir)

print("\nHecho:", [str(p) for p in runs_out])



=== RUN: preset=fast | method=rehearsal | seed=42 | enc=rate | kwargs={'buffer_size': 3000, 'replay_ratio': 0.1} ===

--- Tarea 1/2: circuito1 | preset=fast | method=rehearsal_buf_3000_rr_10 | B=128 T=10 AMP=True | enc=rate ---


[Rehearsal] buffer += None (total=3000/3000) | seen=24981

--- Tarea 2/2: circuito2 | preset=fast | method=rehearsal_buf_3000_rr_10 | B=128 T=10 AMP=True | enc=rate ---
[Rehearsal] mix: task_bs=115 | replay_bs=13 | total_bs=128 | buf_len=3000



=== RUN: preset=fast | method=rehearsal+ewc | seed=42 | enc=rate | kwargs={'buffer_size': 3000, 'replay_ratio': 0.1, 'lam': 1000000000.0, 'fisher_batches': 1000} ===

--- Tarea 1/2: circuito1 | preset=fast | method=rehearsal_buf_3000_rr_10+ewc_lam_1e+09 | B=128 T=10 AMP=True | enc=rate ---


[Rehearsal] buffer += None (total=3000/3000) | seen=24981
[EWC] after_task: estimando Fisher en TRAIN (len=196), cap=1000...
[EWC] Fisher listo: batches_usados=196 | sum=1.533e-01 | max=5.897e-03

--- Tarea 2/2: circuito2 | preset=fast | method=rehearsal_buf_3000_rr_10+ewc_lam_1e+09 | B=128 T=10 AMP=True | enc=rate ---
[Rehearsal] mix: task_bs=115 | replay_bs=13 | total_bs=128 | buf_len=3000



=== RUN: preset=fast | method=as-snn | seed=42 | enc=rate | kwargs={'gamma_ratio': 0.3, 'lambda_a': 1.59168, 'ema': 0.824} ===

--- Tarea 1/2: circuito1 | preset=fast | method=as-snn_gr_0.3_lam_1.59168 | B=128 T=10 AMP=True | enc=rate ---


...

[AS-SNN] α_now=0.2605 | α_ema=0.2618 | γ=0.30 | λ_a=1.59 | pen=0.06083



--- Tarea 2/2: circuito2 | preset=fast | method=as-snn_gr_0.3_lam_1.59168 | B=128 T=10 AMP=True | enc=rate ---


Epoch 1/2:  98%|█████████▊| 83/85 [00:10<00:00,  9.40it/s]

[AS-SNN] α_now=0.1266 | α_ema=0.1274 | γ=0.30 | λ_a=1.59 | pen=0.2748



Hecho: ['/home/cesar/proyectos/TFM_SNN/outputs/continual_fast_rehearsal_buf_3000_rr_10_rate_model-PilotNetSNN_66x200_gray_seed_42', '/home/cesar/proyectos/TFM_SNN/outputs/continual_fast_rehearsal_buf_3000_rr_10+ewc_lam_1e+09_lam_1e+09_rate_model-PilotNetSNN_66x200_gray_seed_42', '/home/cesar/proyectos/TFM_SNN/outputs/continual_fast_as-snn_gr_0.3_lam_1.59168_rate_model-PilotNetSNN_66x200_gray_seed_42']


<a id="sec-07"></a>
## 7) Barrido de combinaciones (opcional)

**Objetivo**  
Explorar matrices de configuración (**presets × seeds × encoders × métodos**) para estudios amplios.

- **Coste alto**: controla `batch_size`, `T` y *workers/prefetch* si la GPU va justa.
- Si usas **offline**, genera los H5 para cada combinación (`encoder/T/gain/size/to_gray`) antes del barrido.
- Mantén **nomenclatura consistente** (el *runner* la parsea después para el resumen).

[↑ Volver al índice](#toc)

In [7]:
# =============================================================================
# Driver de ejecución: barrido de combinaciones (opcional)
# =============================================================================
from copy import deepcopy
from src.runner import run_continual
from src.utils import load_preset, build_task_list_for, build_components_for

PRESETS   = []  # [PRESET] añade "std", "accurate" si quieres
SEEDS     = [CFG["data"]["seed"], 43]
ENCODERS  = [CFG["data"]["encoder"]]
METHODS   = [
    # ("naive", {}),
    # ("ewc", {"lam": 1e9, "fisher_batches": 600}),
    # ("rehearsal", {"buffer_size": 5000, "replay_ratio": 0.2}),
    # ("rehearsal+ewc", {"buffer_size": 5000, "replay_ratio": 0.2, "lam": 7e8, "fisher_batches": 600}),
    ("as-snn", {"gamma_ratio": 0.3, "lambda_a": 1.59168, "ema": 0.824}),
]

for preset_i in PRESETS:
    CFG_i = load_preset(ROOT / "configs" / "presets.yaml", preset_i)
    for seed_i in SEEDS:
        for enc_i in ENCODERS:
            for method_name, method_params in METHODS:
                cfg_i2 = deepcopy(CFG_i)
                cfg_i2["data"]["seed"] = seed_i
                cfg_i2["data"]["encoder"] = enc_i
                cfg_i2["continual"]["method"] = method_name
                cfg_i2["continual"]["params"] = method_params
                if "rehearsal" in method_name:
                    cfg_i2["data"]["persistent_workers"] = False

                tfm_i, make_loader_fn_i, make_model_fn_i = build_components_for(cfg_i2, ROOT)
                task_list_i, tasks_file_i = build_task_list_for(cfg_i2, ROOT)

                print(f"\n=== RUN: preset={preset_i} | method={method_name} | seed={seed_i} "
                      f"| enc={enc_i} | kwargs={method_params} ===")
                out_path, _ = run_continual(
                    task_list=task_list_i,
                    make_loader_fn=make_loader_fn_i,
                    make_model_fn=make_model_fn_i,
                    tfm=tfm_i,
                    cfg=cfg_i2,
                    preset_name=preset_i,
                    out_root=ROOT / "outputs",
                    verbose=True,
                )
                print("OK:", out_path)


<a id="sec-08"></a>
## 8) Resumen completo: inventario → parseo → agregados → tabla

**Objetivo**  
Crear un **resumen reproducible** de todos los *runs*:

- **Inventario** de carpetas `outputs/continual_*`.
- **Parseo** de nombres para extraer `preset`, `método`, `encoder`, `seed`, `modelo`, y parámetros relevantes.
- Cálculo de **olvido** (absoluto y relativo) y **agregados** por grupo (media, σ, n).
- Export a `outputs/summary/continual_summary_agg.csv` y **tabla formateada** para la memoria.

> Si no se detectan *runs*, verifica que exista `continual_results.json` dentro de cada carpeta.  

[↑ Volver al índice](#toc)


In [8]:
# === Resumen de runs (usa utilidades comunes) ===
from pathlib import Path
from src.utils_exp import build_runs_df, aggregate_and_show

outputs_root = ROOT / "outputs"

# (Opcional) inventario rápido en disco
print("Inventario de runs en:", outputs_root)
for p in sorted(outputs_root.glob("continual_*")):
    print(" -", p.name, "| results.json:", (p / "continual_results.json").exists())

df = build_runs_df(outputs_root)
print(f"runs en resumen: {len(df)}")
_ = aggregate_and_show(df, outputs_root)  # también guarda CSV en outputs/summary/


Inventario de runs en: /home/cesar/proyectos/TFM_SNN/outputs
 - continual_accurate_as-snn_gr_0.3_lam_1.59168_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_accurate_ewc_lam_7e+08_lam_7e+08_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_accurate_naive_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_accurate_rehearsal_buf_3000_rr_20+ewc_lam_1e+09_lam_1e+09_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_as-snn_gr_0.3_lam_1.59168_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_as-snn_gr_0.3_lam_1.59168_rate_model-PilotNetSNN_66x200_gray_seed_43 | results.json: True
 - continual_fast_ewc_lam_1e+09_lam_1e+09_ewc_hpo_t0_lam_1.0e+09_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fast_ewc_lam_1e+09_lam_1e+09_ewc_hpo_t2_lam_1.0e+09_rate_model-PilotNetSNN_66x200_gray_seed_42 | results.json: True
 - continual_fas

,exp,preset,method,lambda,encoder,model,seed,c1_name,c2_name,c1_mae,c1_after_c2_mae,c1_forgetting_mae_abs,c1_forgetting_mae_rel_%,c2_mae,lambda_num
0,continual_accurate_as-snn_gr_0.3_lam_1.59168_r...,accurate,as-snn_gr_0.3,1.59168,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.165532,0.200847,0.035314,21.333848,0.217257,1.591680e+00
1,continual_accurate_ewc_lam_7e+08_lam_7e+08_rat...,accurate,ewc,7e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.165532,0.166002,0.000470,0.283758,0.221053,7.000000e+08
2,continual_accurate_naive_rate_model-PilotNetSN...,accurate,naive,None,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.177562,0.239561,0.061999,34.916516,0.220103,NaN
3,continual_accurate_rehearsal_buf_3000_rr_20+ew...,accurate,rehearsal_buf_3000_rr_20+ewc,1e+09,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.173464,0.173046,-0.000419,-0.241270,0.206640,1.000000e+09
4,continual_fast_as-snn_gr_0.3_lam_1.59168_rate_...,fast,as-snn_gr_0.3,1.59168,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.170549,0.174574,0.004025,2.359856,0.225020,1.591680e+00
5,continual_fast_as-snn_gr_0.3_lam_1.59168_rate_...,fast,as-snn_gr_0.3,1.59168,rate,PilotNetSNN_66x200_gray,43,circuito1,circuito2,0.172936,0.172952,0.000017,0.009722,0.224013,1.591680e+00
6,continual_fast_ewc_lam_3e+08_lam_3e+08_rate_mo...,fast,ewc,3e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.172718,0.172754,0.000036,0.020729,0.223894,3.000000e+08
7,continual_fast_ewc_lam_5e+08_lam_5e+08_rate_mo...,fast,ewc,5e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.172764,0.170332,-0.002433,-1.408158,0.222444,5.000000e+08
8,continual_fast_ewc_lam_7e+08_lam_7e+08_rate_mo...,fast,ewc,7e+08,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.170484,0.170454,-0.000029,-0.017053,0.222510,7.000000e+08
9,continual_fast_ewc_lam_1e+09_lam_1e+09_rate_mo...,fast,ewc,1e+09,rate,PilotNetSNN_66x200_gray,42,circuito1,circuito2,0.171993,0.172596,0.000603,0.350517,0.223798,1.000000e+09


Guardado: /home/cesar/proyectos/TFM_SNN/outputs/summary/continual_summary_agg.csv


,preset,método,codificador,λ,MAE Tarea1 (media),Olvido T1 (%) (media),MAE Tarea2 (media),MAE Tarea1 (σ),Olvido T1 (%) (σ),MAE Tarea2 (σ),n (semillas)
0,accurate,as-snn_gr_0.3,rate,1.59168,0.1655,21.3338,0.2173,,,,1
1,accurate,ewc,rate,7e+08,0.1655,0.2838,0.2211,,,,1
2,accurate,naive,rate,NaN,0.1776,34.9165,0.2201,,,,1
3,accurate,rehearsal_buf_3000_rr_20+ewc,rate,1e+09,0.1735,-0.2413,0.2066,,,,1
4,fast,as-snn_gr_0.3,rate,1.59168,0.1717,1.1848,0.2245,0.0017,1.6618,0.0007,2
5,fast,ewc,rate,3e+08,0.1727,0.0207,0.2239,,,,1
6,fast,ewc,rate,5e+08,0.1728,-1.4082,0.2224,,,,1
7,fast,ewc,rate,7e+08,0.1705,-0.0171,0.2225,,,,1
8,fast,ewc,rate,1e+09,0.1720,0.3505,0.2238,,,,1
9,fast,ewc_ewc_hpo_t0_lam_1.0e+09,rate,1e+09,0.1327,3.1007,0.1792,,,,1
